# The BB84 Protocol

In [ ]:
! pip install qiskit
! pip install --upgrade qiskit

! pip install qiskit_aer
! pip install --upgrade qiskit_aer

In [ ]:
import qiskit
import qiskit_aer
import qiskit_ibm_runtime
import random
import math
from typing import Literal

print(f'Qiskit version: {qiskit.__version__}')
print(f'Qiskit Aer version: {qiskit_aer.__version__}')
print(f'Qiskit IBM Runtime version: {qiskit_ibm_runtime.__version__}')

# Global constants
MY_IBM_TOKEN = ''

In [ ]:
class Entity:  # FIXME: Alternative name: `QKD_Party`?
    '''
    Data class for parties involved in QKD protocols.
    '''
    name: str
    a: list[Literal[0, 1]]
    b: list[Literal[0, 1]]

    def __init__(self, name=None, a=None, b=None) -> None:
        self.name = name
        self.a, self.b = a, b


def bernoulli(p: float) -> Literal[0, 1]:
    '''
    Bernoulli distribution RNG. P(X = 0) = p, P(X = 1) = 1 - p.
    '''
    assert 0 <= p <= 1
    return 0 if random.random() < p else 1


def generate_random_bitstring(n: int, p0: float) -> list[Literal[0, 1]]:
    '''
    Generates a random bitstring of length n.
    '''
    assert n > 0
    return list(bernoulli(p0) for _ in range(n))

def my_print(v, s):
    if v:
        print(s)


class BB84:
    '''
    BB84 QKD protocol class.

    Attributes
    ----------
    n (int): Length of random bitstrings\n
    k (float) Fraction of random bits decoded by Bob in the correct basis that are selected for the bit comparison step\n
    alice (Entity): Data object for Alice\n
    bob (Entity): Data object for Bob\n
    eve (Entity): Data object for Eve\n
    has_eavesdropping (bool): whether Eve is doing eavesdropping\n
    qiskit_backend (bool): Run the circuit on this backend\n
    nshots (int): Number of executions of the circuit\n
    '''

    def __init__(self,
        n: int,
        k: float,
        alice: Entity,
        bob: Entity,
        eve: Entity,
        has_eavesdropping: bool = False,
        qiskit_backend = qiskit_aer.AerSimulator(),
        nshots: int = 1,
        verbose: bool = False,
    ) -> None:
        '''
        Set BB84 QKD protocol parameters.
        '''
        assert n > 0
        assert 0 <= k <= 1

        self.n = n; self.k = k
        self.alice = alice; self.bob = bob; self.eve = eve
        self.has_eavesdropping = has_eavesdropping
        self.qiskit_backend = qiskit_backend; self.nshots = nshots
        self.verbose = verbose


    @staticmethod
    def encode(n, a, b, qc):
        qc.barrier(label='resetting >')
        for i in range(n):
            qc.reset(i)  # Reset each qubit
        qc.barrier(label='< resetting')

        qc.barrier(label='encoding >')
        for i in range(n):
            if a[i] == 1:  # Encode the 1-bit with a NOT gate
                qc.x(i)
            if b[i] == 1:  # If the diagonal basis is selected, then apply a HADAMARD gate
                qc.h(i)
        qc.barrier(label='< encoding')
        return qc


    @staticmethod
    def decode(n, b, qc):
        qc.barrier(label='decoding >')
        for i in range(n):
            if b[i] == 1:  # If the diagonal basis is selected, then apply a HADAMARD gate
                qc.h(i)
        qc.barrier(label='< decoding')

        qc.barrier(label='measuring >')
        for i in range(n):
            qc.measure(i, i)  # Apply measurement gates
        qc.barrier(label='< measuring')
        return qc
    

    @staticmethod
    def get_bits(counts: dict[str, int]) -> list[Literal[0, 1]]:
        '''
        Pick the most frequent item from the `counts` frequence table, reverse the sequence order, and map chars to ints.
        '''
        return list(map(int, reversed(max(counts.items(), key=lambda p: p[1])[0])))
    

    @staticmethod
    def run_circuit(qc, backend, nshots) -> dict[str, int]:
        '''
        Execute the quantum circuit on the specified qiskit backend with the specified number of shots.
        '''
        return backend.run(qc, shots=nshots).result().get_counts()


    def __call__(self):
        '''
        1. Alice encodes her random bits into a n-qubit quantum circuit.
            (We opted to send all N qubits together as a quantum circuit instead of sending them one at a time.)
        '''
        qc = qiskit.QuantumCircuit(self.n, self.n, name='BB84 qstate')
        qc = self.encode(self.n, self.alice.a, self.alice.b, qc)

        '''
        2. If eavesdropping is enabled, Eve will intercept the message from Alice to Bob.
        '''
        if self.has_eavesdropping:
            qc         = self.decode(self.n, self.eve.b, qc)
            counts     = self.run_circuit(qc, self.qiskit_backend, self.nshots)
            self.eve.a = self.get_bits(counts)
            qc         = self.encode(self.n, self.eve.a, self.eve.b, qc)

        '''
        3. Bob decodes the random bits from the quantum state (allegedly) received from Alice (or Eve).
        '''
        qc         = self.decode(self.n, self.bob.b, qc)
        counts     = self.run_circuit(qc, self.qiskit_backend, self.nshots)
        self.bob.a = self.get_bits(counts)

        '''
        4. Alice and Bob communicate over the classical channel which qubits were received with the correct basis.
            This is done via bit-by-bit comparison of the bases sequences.
        '''
        bases_sequences_xor = list(enumerate(x ^ y for x, y in zip(self.alice.b, self.bob.b)))
        correct_bits_idx    = list(i for i, x in bases_sequences_xor if not x)

        '''
        5. Alice and Bob compare a fraction k of randomly chosen bits from of the correctly decoded bits over the classical channel.
        '''
        num_comparison_bits = math.floor(self.k * len(correct_bits_idx))  # number of correctly decoded random bits to compare
        comparison_bits_idx = sorted(random.sample(correct_bits_idx, num_comparison_bits))  # sample without replacement the comparison bits
        key_bits_idx        = sorted(list(set(correct_bits_idx) - set(comparison_bits_idx)))  # the remaining correctly decoded bits (that are not used for comparison) will form the secret key

        all_comparison_bits_agree = all(self.alice.a[i] == self.bob.a[i] for i in comparison_bits_idx)  # whether the comparison bits are equal one-by-one

        '''
        6. If all comparison bits agree, then the remaining correct bits are selected as the one-time pad key.
            Eve will use the same indices as Alice and Bob for their key bits.
        '''
        secret_key = eves_key = None

        if not all_comparison_bits_agree:
            my_print(self.verbose, 'QKD failed. Comparison bits did not agree. Try again.')
        elif len(comparison_bits_idx) == 0:
            my_print(self.verbose, 'QKD failed. Too few comparison bits. Try again.')
        elif len(key_bits_idx) == 0:
            my_print(self.verbose, 'QKD failed. Too few key bits. Try again.')
        else:
            secret_key = list(self.bob.a[i] for i in key_bits_idx)
            if self.has_eavesdropping:
                eves_key   = list(self.eve.a[i] for i in key_bits_idx)
            my_print(self.verbose, f'QKD succeeded. Secret key: {secret_key}. Eve\'s key: {eves_key}.')

        # Compile a summary of all relevant values involved in the QKD
        qkd_summary = {
            'Alice random bits sequence':   self.alice.a,
            'Alice bases sequence':         self.alice.b,
            'Bob random bits sequence':     self.bob.a,
            'Bob bases sequence':           self.bob.b,
            'Eve random bits sequence':     self.eve.a,
            'Eve bases sequence':           self.eve.b,

            'Correct bits indices':         correct_bits_idx,
            'Comparison bits indices':      comparison_bits_idx,
            'Agreement of comparison bits': all_comparison_bits_agree,
            'Secret key bits indices':      key_bits_idx,
    
            'Secret key':                   secret_key,
            'Eve\'s key':                   eves_key,

        }

        return secret_key, eves_key, qkd_summary, qc


'''
Set BB84 parameters.
'''
ALICE, BOB, EVE = Entity('Alice'), Entity('Bob'), Entity('Eve')
N = 1
K = 1

ALICE.P0A = 0.5
ALICE.P0B = 0.5
BOB.P0B   = 0.5
EVE.P0B   = 0.5

HAS_EAVESDROPPING = True

QISKIT_BACKEND = qiskit_aer.AerSimulator()
NSHOTS         = 1


'''
Generate random bits sequence for Alice,
    together with bases sequences for both Alice and Bob.
'''
ALICE.a = generate_random_bitstring(N, ALICE.P0A)
ALICE.b = generate_random_bitstring(N, ALICE.P0B)
BOB.b   = generate_random_bitstring(N, BOB.P0B  )
EVE.b   = generate_random_bitstring(N, EVE.P0B  )


'''
Proceed with BB84 QKD protocol.
'''
SECRET_KEY, EVES_KEY, QKD_SUMMARY, QC = BB84(N, K, ALICE, BOB, EVE, HAS_EAVESDROPPING, QISKIT_BACKEND, NSHOTS,)()

QKD_SUMMARY